In [1]:
import pandas as pd
filepath = 'd:/大学/大三上/数据挖掘/作业/数据.xlsx'

In [2]:
data_combine = pd.read_excel(filepath)

In [3]:
data_combine_copy = data_combine.copy

In [4]:
import nltk
from nltk.corpus import stopwords

In [5]:
data_combine['word_split'] = data_combine.text.apply(lambda x:nltk.word_tokenize(str(x).lower()))

In [6]:
def dv_sa(x):
    a= ''
    for i in x:
        if i not in  stopwords.words('english'):
            i = '*' 
        a = a+' '+i
    return a

data_combine['dv_sa']=data_combine.word_split.apply(lambda x: dv_sa(x))

In [7]:
data_combine.dv_sa

0         this * * * * * me no * of * the * of my * * a...
1                it * once * to me that the * * be a * * *
2         in his * * was a * * * * from which * as he *...
3         how * is * as we * from * * on the * * * * * ...
4         * * * * not * * * the * * his * * but a * * *...
5         a * * in * * my * * * under your * and * * * ...
6         the * * * * at this * * * * in the * of * * *...
7                                 the * * in * from my * *
8         i * that you * not * to yourself * * * being ...
9         i * that * the * of * * nor the * of * * nor ...
10        he * * that i can * my * * he * * to * my * *...
11          here we * ourselves * and * for the * were * *
12          * * * * * because his * * was the * of the * *
13              the * * * * * very * up the * * * to * * *
14                        but a * will * the * of this * *
15        he had * me * and i * * a * and * * * * the *...
16        to these * they * * of * * their own * * * * .

In [24]:
train_data = data_combine[:-8392]
test_data = data_combine[-8392:]

In [8]:
from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [9]:
tfidf = TfidfVectorizer(ngram_range=(1,5),analyzer='word',token_pattern='.*? .*?')
# 写一个正则匹配就行了

In [10]:
tfidf.fit(data_combine.dv_sa.values.tolist())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 5), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='.*? .*?', tokenizer=None, use_idf=True,
        vocabulary=None)

In [16]:
from sklearn.decomposition import TruncatedSVD

In [17]:
svd = TruncatedSVD(n_components=25,algorithm='arpack')

In [18]:
svd_matrix = svd.fit_transform(tfidf.transform(data_combine.dv_sa.values.tolist()))

In [19]:
print(svd_matrix)

[[ 0.41728845 -0.05932833 -0.00514919 ...,  0.01895476  0.06369608
  -0.03023372]
 [ 0.12905421 -0.03250405 -0.03129884 ..., -0.01946209  0.00700708
  -0.02436234]
 [ 0.39316539  0.01485718  0.02067984 ..., -0.00185167  0.02376045
  -0.02453574]
 ..., 
 [ 0.29190046  0.06774227 -0.04394066 ..., -0.01476449 -0.0065136
   0.03927061]
 [ 0.29888564 -0.06875855 -0.11935415 ..., -0.0308416   0.00662439
  -0.00910301]
 [ 0.35595373  0.02624089  0.00068273 ...,  0.02726556 -0.02396378
  -0.00951804]]


In [20]:
svd_feature = pd.DataFrame(svd_matrix)

In [21]:
svd_feature.columns = ['svd_word_1to5gram_'+str(i)for i in range(25)]

In [22]:
svd_feature

,svd_word_1to5gram_0,svd_word_1to5gram_1,svd_word_1to5gram_2,svd_word_1to5gram_3,svd_word_1to5gram_4,svd_word_1to5gram_5,svd_word_1to5gram_6,svd_word_1to5gram_7,svd_word_1to5gram_8,svd_word_1to5gram_9,...,svd_word_1to5gram_15,svd_word_1to5gram_16,svd_word_1to5gram_17,svd_word_1to5gram_18,svd_word_1to5gram_19,svd_word_1to5gram_20,svd_word_1to5gram_21,svd_word_1to5gram_22,svd_word_1to5gram_23,svd_word_1to5gram_24
0,0.417288,-0.059328,-0.005149,-0.105973,0.017783,0.062779,-0.000788,-0.030995,0.000617,0.006312,...,-0.021836,-0.015172,-0.005943,0.021225,0.006510,-0.008601,-0.049341,0.018955,0.063696,-0.030234
1,0.129054,-0.032504,-0.031299,-0.012039,-0.026727,-0.028694,-0.005784,0.028315,-0.016790,-0.031917,...,-0.014272,-0.012627,-0.010698,0.004665,0.011106,-0.031824,0.020486,-0.019462,0.007007,-0.024362
2,0.393165,0.014857,0.020680,0.041938,-0.077190,-0.074827,0.006063,-0.036990,-0.029500,0.088121,...,-0.007205,0.137822,-0.009411,-0.053189,-0.013259,0.032555,-0.017218,-0.001852,0.023760,-0.024536
3,0.431843,0.170288,0.012018,0.056804,0.060088,0.029557,0.052073,0.008505,-0.033771,-0.030830,...,-0.003336,-0.013759,-0.004596,0.006523,-0.034179,-0.016593,-0.069720,-0.054044,0.048533,0.130747
4,0.371597,0.056731,-0.030600,-0.000377,0.020538,-0.171411,-0.075223,-0.060892,-0.028735,0.158025,...,0.038305,-0.040860,0.006561,0.018366,-0.020379,-0.024971,-0.007429,-0.018509,-0.015108,-0.017319
5,0.410738,-0.069376,-0.131510,-0.014386,0.009195,0.015731,-0.001034,-0.030047,-0.035990,0.032224,...,0.037012,-0.042748,-0.005660,0.037750,-0.020700,-0.013606,0.008315,-0.006100,0.085094,0.010299
6,0.420736,0.024556,0.056259,0.044619,-0.054392,-0.019244,0.067174,0.038590,0.129826,0.005769,...,0.038184,0.052318,-0.008065,0.040774,0.012544,-0.021630,-0.033037,-0.007016,-0.057448,0.011459
7,0.161551,-0.062908,-0.000899,-0.030362,-0.005816,-0.015325,0.056839,0.043435,0.034160,0.009338,...,0.042693,-0.008407,0.023774,0.023221,-0.029628,0.064831,-0.043928,0.116593,0.040899,0.034445
8,0.419964,-0.073666,-0.071886,-0.045761,0.010040,0.046663,-0.019304,-0.028521,-0.015125,-0.032041,...,0.001321,0.025708,0.023210,0.014258,0.015905,-0.017824,0.017192,0.029709,0.011021,-0.041745
9,0.235980,-0.065944,0.031043,0.017188,-0.080997,0.086109,-0.093950,-0.087403,0.117280,0.000632,...,-0.027739,-0.015240,-0.035863,0.023650,0.058213,0.010156,-0.000735,-0.018656,0.021169,0.037469


In [23]:
full_idf = tfidf.fit_transform(data_combine.dv_sa.values.tolist())
train_idf = tfidf.transform(train_data.dv_sa.values.tolist())
test_idf = tfidf.transform(test_data.dv_sa.values.tolist())

NameError: name 'train_data' is not defined

In [28]:
from sklearn.model_selection import KFold

In [44]:
svd_matrix_ = svd.fit_transform(tfidf_.transform(data_combine.dv_sa.values.tolist()))

In [25]:
train_data = data_combine[:-8392]
test_data = data_combine[-8392:]

In [26]:
full_idf = tfidf.fit_transform(data_combine.dv_sa.values.tolist())
train_idf = tfidf.transform(train_data.dv_sa.values.tolist())
test_idf = tfidf.transform(test_data.dv_sa.values.tolist())

In [29]:
import numpy as np
cv_score = []
pred_full_test = 0
pred_train = np.zeros([train_data.shape[0],3])
kf = KFold(n_splits=5,shuffle=True,random_state=2017)

In [30]:
from sklearn.naive_bayes import MultinomialNB
def run_by(train_x,train_y,test_x,test_y,test_x2):
    model = MultinomialNB()
    model.fit(train_x,train_y)
    pred_test_y = model.predict_proba(test_x)
    preb_test_y2 = model.predict_proba(test_x2)
    return pred_test_y,preb_test_y2,model

In [31]:
author = {'EAP':0,
         'HPL':1,
         'MWS':2}
train_y = train_data['author'].map(author)

In [32]:
from sklearn import metrics

In [34]:
for dev_index,val_index in kf.split(train_y):
    dev_x,val_x = train_idf[dev_index],train_idf[val_index]
    dev_y,val_y = train_y[dev_index],train_y[val_index]
    pred_val_y,pred_test_y,model = run_by(dev_x,dev_y,val_x,val_y,test_idf)
    pred_full_test+=pred_test_y
    pred_train[val_index,:] = pred_val_y
    cv_score.append(metrics.log_loss(val_y,pred_val_y))
print(cv_score)
pred_full_test/=5

[0.950374684619815, 0.93509830946931305, 0.93873820881282244, 0.95120680248155609, 0.95034412058227014]


In [134]:
train_data = data_combine[:-8392]
test_data = data_combine[-8392:]

In [35]:
train_data['eap_svd_word_dv_feature'] = pred_train[:,0]
train_data['hpl_svd_word_dv_feature'] = pred_train[:,1]
train_data['mws_svd_word_dv_feature'] = pred_train[:,2]
test_data['eap_svd_word_dv_feature'] = pred_full_test[:,0]
test_data['hpl_svd_word_dv_feature'] = pred_full_test[:,1]
test_data['mws_svd_word_dv_feature'] = pred_full_test[:,2]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [36]:
data_combine = pd.concat([train_data,test_data],axis=0)

In [38]:
data_combine

,author,id,text,word_split,text_len,symbol_num,"symbol_num_,",symbol_num_.,symbol_num_?,symbol_num_:,...,eap_svd_char_feature,hpl_svd_char_feature,mws_svd_char_feature,eap_svd_word_feature,hpl_svd_word_feature,mws_svd_word_feature,dv_sa,eap_svd_word_dv_feature,hpl_svd_word_dv_feature,mws_svd_word_dv_feature
0,EAP,id26305,"This process, however, afforded me no means of...","[this, process, ,, however, ,, afforded, me, n...",48,5,4,1,0,0,...,0.768579,0.104705,0.126715,0.699703,0.042168,0.258129,this * * * * * me no * of * the * of my * * a...,0.929799,0.018097,0.052104
1,HPL,id17569,It never once occurred to me that the fumbling...,"[it, never, once, occurred, to, me, that, the,...",15,1,0,1,0,0,...,0.357804,0.347917,0.294279,0.536703,0.218901,0.244396,it * once * to me that the * * be a * * *,0.565063,0.289390,0.145547
2,EAP,id11008,"In his left hand was a gold snuff box, from wh...","[in, his, left, hand, was, a, gold, snuff, box...",41,5,4,1,0,0,...,0.268405,0.664779,0.066816,0.735030,0.088748,0.176222,in his * * was a * * * * from which * as he *...,0.524806,0.393330,0.081863
3,MWS,id27763,How lovely is spring As we looked from Windsor...,"[how, lovely, is, spring, as, we, looked, from...",38,4,3,1,0,0,...,0.081912,0.687833,0.230255,0.090574,0.046155,0.863270,how * is * as we * from * * on the * * * * * ...,0.478825,0.295736,0.225438
4,HPL,id12958,"Finding nothing else, not even gold, the Super...","[finding, nothing, else, ,, not, even, gold, ,...",31,3,2,1,0,0,...,0.450563,0.388016,0.161420,0.415742,0.417151,0.167107,* * * * not * * * the * * his * * but a * * *...,0.457001,0.346954,0.196045
5,MWS,id22965,"A youth passed in solitude, my best years spen...","[a, youth, passed, in, solitude, ,, my, best, ...",90,6,4,1,0,1,...,0.056302,0.004886,0.938812,0.089834,0.167006,0.743160,a * * in * * my * * * under your * and * * * ...,0.584400,0.075011,0.340589
6,EAP,id09674,"The astronomer, perhaps, at this point, took r...","[the, astronomer, ,, perhaps, ,, at, this, poi...",26,4,3,1,0,0,...,0.395549,0.486149,0.118302,0.922745,0.032733,0.044522,the * * * * at this * * * * in the * of * * *...,0.834214,0.083962,0.081824
7,EAP,id13515,The surcingle hung in ribands from my body.,"[the, surcingle, hung, in, ribands, from, my, ...",9,1,0,1,0,0,...,0.557539,0.153258,0.289202,0.759421,0.185771,0.054808,the * * in * from my * *,0.698317,0.120564,0.181119
8,EAP,id19322,I knew that you could not say to yourself 'ste...,"[i, knew, that, you, could, not, say, to, your...",98,9,7,1,0,0,...,0.053949,0.008103,0.937947,0.396658,0.481202,0.122140,i * that you * not * to yourself * * * being ...,0.726026,0.040409,0.233564
9,MWS,id00912,I confess that neither the structure of langua...,"[i, confess, that, neither, the, structure, of...",26,3,2,1,0,0,...,0.541304,0.210182,0.248514,0.545770,0.174261,0.279969,i * that * the * of * * nor the * of * * nor ...,0.431769,0.208978,0.359253


In [39]:
for i in range(25):
    data_combine['svd_word_dv_1to5gram_'+str(i)] = svd_feature['svd_word_1to5gram_'+str(i)]

In [40]:
data_combine

,author,id,text,word_split,text_len,symbol_num,"symbol_num_,",symbol_num_.,symbol_num_?,symbol_num_:,...,svd_word_dv_1to5gram_15,svd_word_dv_1to5gram_16,svd_word_dv_1to5gram_17,svd_word_dv_1to5gram_18,svd_word_dv_1to5gram_19,svd_word_dv_1to5gram_20,svd_word_dv_1to5gram_21,svd_word_dv_1to5gram_22,svd_word_dv_1to5gram_23,svd_word_dv_1to5gram_24
0,EAP,id26305,"This process, however, afforded me no means of...","[this, process, ,, however, ,, afforded, me, n...",48,5,4,1,0,0,...,-0.021836,-0.015172,-0.005943,0.021225,0.006510,-0.008601,-0.049341,0.018955,0.063696,-0.030234
1,HPL,id17569,It never once occurred to me that the fumbling...,"[it, never, once, occurred, to, me, that, the,...",15,1,0,1,0,0,...,-0.014272,-0.012627,-0.010698,0.004665,0.011106,-0.031824,0.020486,-0.019462,0.007007,-0.024362
2,EAP,id11008,"In his left hand was a gold snuff box, from wh...","[in, his, left, hand, was, a, gold, snuff, box...",41,5,4,1,0,0,...,-0.007205,0.137822,-0.009411,-0.053189,-0.013259,0.032555,-0.017218,-0.001852,0.023760,-0.024536
3,MWS,id27763,How lovely is spring As we looked from Windsor...,"[how, lovely, is, spring, as, we, looked, from...",38,4,3,1,0,0,...,-0.003336,-0.013759,-0.004596,0.006523,-0.034179,-0.016593,-0.069720,-0.054044,0.048533,0.130747
4,HPL,id12958,"Finding nothing else, not even gold, the Super...","[finding, nothing, else, ,, not, even, gold, ,...",31,3,2,1,0,0,...,0.038305,-0.040860,0.006561,0.018366,-0.020379,-0.024971,-0.007429,-0.018509,-0.015108,-0.017319
5,MWS,id22965,"A youth passed in solitude, my best years spen...","[a, youth, passed, in, solitude, ,, my, best, ...",90,6,4,1,0,1,...,0.037012,-0.042748,-0.005660,0.037750,-0.020700,-0.013606,0.008315,-0.006100,0.085094,0.010299
6,EAP,id09674,"The astronomer, perhaps, at this point, took r...","[the, astronomer, ,, perhaps, ,, at, this, poi...",26,4,3,1,0,0,...,0.038184,0.052318,-0.008065,0.040774,0.012544,-0.021630,-0.033037,-0.007016,-0.057448,0.011459
7,EAP,id13515,The surcingle hung in ribands from my body.,"[the, surcingle, hung, in, ribands, from, my, ...",9,1,0,1,0,0,...,0.042693,-0.008407,0.023774,0.023221,-0.029628,0.064831,-0.043928,0.116593,0.040899,0.034445
8,EAP,id19322,I knew that you could not say to yourself 'ste...,"[i, knew, that, you, could, not, say, to, your...",98,9,7,1,0,0,...,0.001321,0.025708,0.023210,0.014258,0.015905,-0.017824,0.017192,0.029709,0.011021,-0.041745
9,MWS,id00912,I confess that neither the structure of langua...,"[i, confess, that, neither, the, structure, of...",26,3,2,1,0,0,...,-0.027739,-0.015240,-0.035863,0.023650,0.058213,0.010156,-0.000735,-0.018656,0.021169,0.037469


In [41]:
writer = pd.ExcelWriter('D:/大学/大三上/数据挖掘/作业/数据.xlsx')
data_combine.to_excel(writer,sheet_name='sheet1')
writer.save()